In [ ]:
import cupy as cp
import cudf
import cuml
import glob
from tqdm import tqdm

In [ ]:
path = '/kaggle/input/optiver-realized-volatility-prediction'
trade_train_path = glob.glob(f'{path}/trade_train.parquet/*/*.parquet')
book_train_path = glob.glob(f'{path}/book_train.parquet/*/*.parquet')

In [ ]:
def get_minmax_shape(book_train_path):
    for k, v in enumerate(book_train_path):
        shape = read_stock_id(v) \
            [['time_id', 'seconds_in_bucket']] \
            .pivot(['time_id'], 'seconds_in_bucket').shape
        if k == 0:
            max_len = shape[0]
            max_width = shape[1]
            min_len = shape[0]
            min_width = shape[1]
        if k > 0:
            if shape[0] > max_len:
                max_len = shape[0]
            if shape[1] > max_width:
                max_width = shape[1]
            if shape[0] < min_len:
                min_len = shape[0]
            if shape[1] < min_width:
                min_width = shape[1]
                
    return {'max':(max_len, max_width), 'min':(min_len, min_width)}

get_minmax_shape(book_train_path)

In [ ]:
from numba import cuda, float32, float64
import math

def cu_std_transform(x, y_out):
    res = cuda.shared.array(1, dtype=float64)
    res[0] = 0
    cuda.syncthreads()

    for i in range(cuda.threadIdx.x, len(x), cuda.blockDim.x):
        cuda.atomic.add(res, 0, x[i])
    cuda.syncthreads()

    for i in range(cuda.threadIdx.x, len(x), cuda.blockDim.x):
#         y_out[i] = ((abs(res[0] - (res[0] / len(x)))**2)/len(x))**0.5
        y_out[i] = (abs(res[0] - (res[0] / len(x)))**2/len(x))**0.5

def cu_mean_transform(x, y_out):
    res = cuda.shared.array(1, dtype=float32)
    res[0] = 0
    cuda.syncthreads()

    for i in range(cuda.threadIdx.x, len(x), cuda.blockDim.x):
        cuda.atomic.add(res, 0, x[i])
    cuda.syncthreads()

    for i in range(cuda.threadIdx.x, len(x), cuda.blockDim.x):
        y_out[i] = res[0] / len(x)

def get_cu_shift_transform(shift_by, null_val=-1):
    def cu_shift_transform(x, y_out):
        for i in range(cuda.threadIdx.x, len(x), cuda.blockDim.x):
            y_out[i] = null_val
            if 0 <= i-shift_by < len(x):
                y_out[i] = x[i-shift_by]
    return cu_shift_transform

In [ ]:
data = read_stock_id(book_train_path[0]).query("time_id==5")#.set_index(['time_id', 'seconds_in_bucket'])
data = time_id_map.join(data, how='inner', sort=True)
data = cudf.DataFrame(data.to_pandas().fillna(method='ffill')).reset_index()


In [ ]:
%cd /kaggle/input/rapids-kaggle-utils/

In [ ]:
# import cu_utils.transform as cutran
import numpy as np

read_stock_id = lambda f: cudf.read_parquet(f)

time_id_map = read_stock_id(book_train_path[0]) \
            [['time_id', 'seconds_in_bucket']] \
            .pivot(['time_id'], 'seconds_in_bucket')['time_id'] \
            .stack(dropna=False) \
            .to_frame().drop(0,1).reset_index() \
            .rename({0:'time_id', 1:'seconds_in_bucket'}, axis=1) \
            .set_index(['time_id', 'seconds_in_bucket'])

means = lambda col: data.groupby('time_id', method='cudf').apply_grouped(
            cu_mean_transform,
            incols={col: 'x' for c in data.columns},
            outcols={'y_out': cp.float32},
            tpb=32
        )['y_out']

stds = lambda col: data.groupby('time_id', method='cudf').apply_grouped(
            cu_std_transform,
            incols={col: 'x' for c in data.columns},
            outcols={'y_out': cp.float32},
            tpb=32
        )['y_out']

standardize = lambda col: (data[col] - means(col))/stds(col)

def diff_log_wap_shifted(n,b):
    logwap = lambda bp, ap, bz, az : (((bp*az)+(ap*bz))/(bz+az)).log() 
    cols = [b[f'{col}{n}'] for col in ['bid_price', 'ask_price', 'bid_size', 'ask_size']]
    b[f'log_wap{n}'] = logwap(*cols)
    log_wap_shifted = b.groupby(['time_id'], method='cudf') \
        .apply_grouped(
            get_cu_shift_transform(shift_by=1, null_val=np.NaN),
            incols={f'log_wap{n}': 'x'},
            outcols={'y_out': cp.float32},
            tpb=32
        )['y_out']
    return (b[f'log_wap{n}'] - log_wap_shifted)**2

def get_logret(b):
    b[f'logret1'] = diff_log_wap_shifted(1,b)
    b[f'logret2'] = diff_log_wap_shifted(2,b)
    return b


def getvol(f): 
    b = cudf.read_parquet(f)     
    b = read_stock_id(f).set_index(['time_id', 'seconds_in_bucket'])
    b = time_id_map.join(b, how='left', sort=True)
    b = cudf.DataFrame(b.to_pandas().fillna(method='ffill')).reset_index()
   
#     for col in b.columns:
#         if col not in ['time_id', 'seconds_in_bucket']:
#             b[col] = standardize(col)
        
    b['stock_id'] = f.split('/')[-2].split('=')[-1]
    b = get_logret(b)

    b = b.query(f"logret1==logret1")

    b = b.groupby(['stock_id','time_id']).agg({
        'logret1': {'sum', 'std'}, 
        'logret2': {'sum', 'std'}, 
        'time_id':'count'})
    
    b.columns = b.columns.map('_'.join)
    sum_cols = [i for i in b.columns if 'sum' in i]
    b[[f'vol{k+1}' for k,_ in enumerate(sum_cols)]] = b[sum_cols]**0.5
    b['volrate'] = b.vol1 / b.vol2
    return b.reset_index()

In [ ]:
vols = cudf.concat([getvol(f) for f in book_train_path]) \
    .set_index(['stock_id', 'time_id'])
vols

In [ ]:
from cuml.metrics.regression import r2_score
rmspe_lambda = lambda y_true, y_pred: np.mean(((y_true - y_pred) / y_true)**2)**0.5

train_path = glob.glob(f'{path}/train.csv')[0]
train = cudf.read_csv(train_path).set_index(['stock_id', 'time_id'])
features = ['vol1', 'vol2', 'logret1_std', 'logret2_std', 'volrate', 'time_id_count']
m = train.join(vols.fillna(0), how='inner')[['target']+features].reset_index()
rr = round(r2_score(m.target, m.vol1), 3)
rmspe = round(rmspe_lambda(m.target, m.vol1), 3)
print(rr, rmspe)

In [ ]:
# cutran.cu_min_transform()
# for i in book_train_path:
# t = read_stock_id(i)
# t_mean_trans = t.groupby('time_id', method='cudf').apply_grouped(
#             cutran.cu_mean_transform,
#             incols={f'seconds_in_bucket': 'x'},
#             outcols={'y_out': cp.float32},
#             tpb=32
#         )['y_out']


# if len(t[t_min_trans!=0]) > 0:
#     print(i)
#     x = t[t_min_trans!=0]
#     break


In [ ]:
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV

xgbreg = XGBRegressor(tree_method='gpu_hist', gpu_id=0)

parameters = {
        'predictor': ['gpu_predictor'],
        'objective':['reg:squarederror', 'reg:logistic', 'reg:squaredlogerror'],
        'eval_metric': ['rmse'],
        'booster': ['gblinear'],
        'subsample': [0.75, 0.5, 0.25],
        'n_estimators': [750, 1000]}

xgb_grid = GridSearchCV(xgbreg, parameters, cv=3, verbose=False, n_jobs=-1)

In [ ]:
# x_train_full = cp.array(m[['stock_id', 'time_id']+features].to_gpu_matrix()).get()
# y_train_full = cp.array(m[['target']].to_gpu_matrix()).get()
# xgb_grid.fit(x_train_full, y_train_full)

In [ ]:
# print(xgb_grid.best_score_)
# print(xgb_grid.best_params_)

In [ ]:
# %cd /kaggle/working

In [ ]:
# df.to_csv("submission.csv", index=False, columns=["row_id", "target"])

In [ ]:
# cudf.read_csv("submission.csv")